In [ ]:
%matplotlib widget
import ibvpy.api as ib
import matplotlib.pylab as plt
from matplotlib import cm
from scipy.interpolate import interp2d
from scipy.signal import argrelextrema
from bmcs_shear.api import CrackBridgeAdv
from bmcs_shear.dic_crack import\
    DICGrid, DICStateFields, DICCrackList
import numpy as np
np.seterr(divide='ignore', invalid='ignore');

In [ ]:
dic_grid = DICGrid(start_t=0, end_t=1, U_factor=100, dir_name='B9_TV1_')

In [ ]:
dsf = DICStateFields(dic_grid=dic_grid, t=0.5)
dsf.tmodel_.trait_set(E=5000, c_T=0, nu=0.18, epsilon_0=0.0005, epsilon_f=0.01);
dsf.eval()

In [ ]:
dcl = DICCrackList(dsf=dsf)

# Identify the time index corresponding to 75 % of the ultimate load

**Remark:** do it in dic_grid

# Identify the cracks that cross the mid height of the section

In [ ]:
x_NC, y_NC, N_tip_C, M_NC = dcl.primary_cracks

In [ ]:
n_N, n_C = M_NC.shape

In [ ]:
mid_N = int(n_N / 3)

In [ ]:
PC_C = np.where(N_tip_C >= mid_N)[0]
PC_C

In [ ]:
M_mid_C = M_NC[mid_N, PC_C]
M_mid_C

In [ ]:
x, M, dM = np.unique(M_mid_C, return_index=True, return_counts=True)
PC_C[M], dM

To extract the opening and sliding displacements along the crack path, let us
construct an interploator which takes the input displacements $x_{IJ}$.
Given the line of the crack x(s), y(s) find a normal to the crack from the point $x_{IJ}$

In [ ]:
PC_C[M] + dM - 1

In [ ]:
def get_f_ironed(x, y, r=10):
    '''Averaging of a function using a bell-shaped ironing function
    '''
    RR = r
    n = len(x)
    delta_x = x[None, ...] - x[..., None]
    r2_n = (delta_x ** 2) / (2 * RR ** 2)
    alpha_r_MJ = np.exp(-r2_n)
    a_M = np.trapz(alpha_r_MJ, x, axis=-1)
    normed_a_MJ = np.einsum('MJ,M->MJ', alpha_r_MJ, 1 / a_M)
    y_MJ = np.einsum('MJ,J->MJ', normed_a_MJ, y)
    y_smooth = np.trapz(y_MJ, x, axis=-1)
    return y_smooth

In [ ]:
from scipy.interpolate import CubicSpline, UnivariateSpline

In [ ]:
C = 4
R = 20
x_C, y_C = x_NC[:, C], y_NC[:, C]
x_C_irn = get_f_ironed(y_C, x_C, R)

In [ ]:
cs = CubicSpline(y_C[:N_tip_C[C]+3], x_C_irn[:N_tip_C[C]+3], bc_type='clamped')

In [ ]:
fig, ax = plt.subplots(1,1)
#ax.plot(x_C_irn[:tip_NC[C]], y_C[:tip_NC[1]], color='orange')
ax.plot(x_C, y_C, 'o')
y_range = np.linspace(y_C[0], y_C[N_tip_C[C]+2], 20)
ax.plot(cs(y_range), y_range, color='green');
ax.axis('equal');

In [ ]:
d_x = 30
X_right = np.array([cs(y_range) + d_x, y_range], dtype=np.float_).T
X_left = np.array([cs(y_range) - d_x, y_range], dtype=np.float_).T
U_C = dsf.interp_U(X_right) - dsf.interp_U(X_left)
U_C

# Identify the index ranges corresponding to the cracks

In [ ]:
xx_MN, yy_MN, cd_field_irn_MN = dsf.crack_detection_field

In [ ]:
nn_M, nn_N = xx_MN.shape

In [ ]:
arg_x_NP = arg_x_NC[:,primary_cracks]

In [ ]:
first_x_NP = np.zeros((mid_N,1), dtype=np.int_)
last_x_NP = np.zeros((mid_N,1), dtype=np.int_) + nn_M
arg_mid_NP = np.hstack([ 
    first_x_NP, arg_x_NP[:mid_N,:], last_x_NP
])
arg_mid_NP

In [ ]:
# BNT - boundary, row, tooth
teeths_BNT = np.array([
    arg_mid_NP[:, :-1],
    arg_mid_NP[:, 1:]
]
)
teeths_TNB = np.einsum('BNT->TNB', teeths_BNT)

In [ ]:
teeths_TNB[0, :, 0] + 3

In [ ]:
teeth_list_aP = []
for tooth_NB in teeths_TNB[1:]:
    tooth_aP = [
        np.array([np.arange(*tooth_B),
                  np.ones(tooth_B[1]-tooth_B[0])*row],
                 dtype=np.int_)
        for row, tooth_B in enumerate(tooth_NB)
    ]
    teeth_list_aP.append(np.hstack(tooth_aP))

In [ ]:
teeth_list_aP[-1]

In [ ]:
xx_MN[teeth_list_aP[2][0], teeth_list_aP[2][1]][::10]

In [ ]:
arg_x_NP.shape

# Get the indexes along the crack ligament

In [ ]:
arg_x_NP_left = arg_x_NP[:, :, np.newaxis] + np.arange(-2, 0)[np.newaxis, np.newaxis, :];
arg_x_NP_right = arg_x_NP[:, :, np.newaxis] + np.arange(1, 3)[np.newaxis, np.newaxis, :]

In [ ]:
arg_x_NP_left[0,0,:], arg_x_NP_right[0,0,:]

In [ ]:
arg_x_NP[0,0]

The indexes are obtained on a resolution xx - i.e. the grid used for interpolation of 
the crack detection field.

- the evaluation of sliding and opening makes only sense in the localized range
- construct an `xx` interpolation of the displacement field over the whole domain.
- construct the crack path and smooth it
- construct the ortho-normal bases along the crack path (available in crack-path object)
- discretize along the normal direction within a predefined range
- pick up the global displacements $U_{na}$ along the normal vectors
- use the ortho-normal bases to transform the displacements into the local coordinate system
- center the displacement vector relative to the displacement at the crack position
- find the maximum difference in tangential and normal directions 

# How to define the fixed and rotating frames

Is the current implementation applicable also for the aligned grid analysis?
Actually, it should and it should.